Below we supplied two examples of basic MPI code. There are no official excercises but feel free to whip up some mock data and try out the code provided.

**Serial Histogram**


In [0]:
def serial_histogram(data,binscount):
    results=[0]*binscount
    mindata=min(data)
    maxdata=max(data)
    print (mindata,maxdata)
    binsize=(maxdata-mindata)/binscount
    binlabels=[]
    
    histogram=[0]*(binscount)
    for i in range(0,binscount):
        binlabels.append(mindata+(i*(binsize)))
    for i in range(0,data.size):
        index=((data[i]-mindata)/(maxdata-mindata))*(binscount-1)
        
        histogram[int(index)]= histogram[int(index)]+1
    return binlabels,histogram 

**An Extended example of MPI**

Due to size contraints it was not possible to upload the data cube!

In [0]:
from mpi4py import MPI
import numpy
import math


def Calculate_Star_End_Channels(FullDataCube,psize,currentprank):
    ZDim=FullDataCube.shape[0]
    NumerofChannels=math.ceil(ZDim/float(psize))
    StartChannel=min(int((currentprank/float(psize))*ZDim),ZDim) 
    EndChannel=min(int(((currentprank+1)/float(psize))*ZDim),ZDim) 
    print ("%d: from %d to %d\n" % (currentprank,StartChannel,EndChannel))
    return StartChannel,EndChannel
    

def GetLocalDataCube(FilePath,psize,currentprank):
    hdulist = fits.open(FilePath)
    ## Convert it to 3D data only
    datacube=hdulist[0].data.squeeze()
    ## Get How many spectral channels
    StartChannel,EndChannel=Calculate_Star_End_Channels(datacube,mpi_size,mpi_rank)    
    LocalDataCube=datacube[StartChannel:EndChannel,:,:].ravel()    
    return LocalDataCube

def CalculateBinLabels(bin_count,bin_Size):
    binlabels=[]
    for i in range(0,bin_count):
        binlabels.append(global_min+(i*(bin_Size)))
    return binlabels

def CalculateLocalHist(LocalDataCube,global_min,global_max,bins_count):
   histogram=[0]*(binscount)
   for i in range(0,LocalDataCube.size):
       index=((LocalDataCube[i]-global_min)/(global_max-global_min))*(bins_count-1)        
       histogram[int(index)]= histogram[int(index)]+1
   return histogram
    

mpi_comm = MPI.COMM_WORLD
mpi_rank = mpi_comm.Get_rank()
mpi_size = mpi_comm.Get_size()



LocalDataCube=GetLocalDataCube('/data/fig7.fits',mpi_size,mpi_rank)

## Calculate Local Min and Local Max
localmin=min(LocalDataCube)
localmax=max(LocalDataCube)

## Get the global Min and Max
global_min=mpi_comm.reduce(localmin,op=MPI.MIN,root=0)
global_max=mpi_comm.reduce(localmax,op=MPI.MAX,root=0)

## Send the Global Min and Max to all Ranks --- You will need it 
global_min=mpi_comm.bcast(global_min,root=0)
global_max=mpi_comm.bcast(global_max,root=0)


binscount=10
binsize=(global_max-global_min)/binscount

    

if mpi_rank==0:    
    print CalculateBinLabels(binscount,binsize)
    
    
histogram=CalculateLocalHist(LocalDataCube,global_min,global_max,binscount)

if mpi_rank ==0:
    for j in range(1,mpi_size):
        Recv_histogram = mpi_comm.recv(source=MPI.ANY_SOURCE, tag=1)
        for i in range(0,binscount):
            histogram[i]=histogram[i]+ Recv_histogram[i] 
    print (histogram)
else:
    mpi_comm.send(histogram,dest=0,tag=1)
    